In [ ]:
import os
import imageio as iio
from pathlib import Path
import numpy as np
from glob import glob
import matplotlib.cm as cm
import matplotlib.pyplot as plt

#AIRXD model import
from airxd.mask import MASK
from airxd_cnn.cnn import ARIXD_CNN as cmodel
from airxd.model import ARIXD
from airxd.dataset import Dataset, parse_imctrl

# Troubleshooting

In [ ]:
import skimage
from skimage.morphology import disk
from skimage.filters import rank
from skimage import exposure

def normalize(image):
        footprint = disk(32)
        img = np.log(np.abs(image) - np.min(image) + 1e-7)
        p2, p98 = np.percentile(img, (2, 98))
        img = exposure.rescale_intensity(img, in_range=(p2, p98))
        img = skimage.util.img_as_ubyte(img)
        img_eq = rank.equalize(img, footprint)
        img_eq = img_eq.astype(float)/256.0
        return img_eq  

In [ ]:
from PIL import Image

In [ ]:
import time

In [ ]:
temp_path = 'data/Nickel/'
images = glob(temp_path + '*.tif')

i = 0

#Time the code below
start = time.time()

test_im = iio.v2.volread(images[i])
test_im_norm = normalize(test_im)


#Stop recording time
end = time.time()
#Plot positions of all nonzero elements in test_im
plt.figure(figsize=(10,10))
plt.imshow(test_im_norm, cmap='binary', origin='lower')
plt.title('Raw data')
plt.axis('off')
plt.tight_layout()
plt.show()

print(f'Time to load image: {end-start} seconds')

# Create/load a specific dataset

Here, we'll be utilizing the auto-masking process developed in __ to automatically generate labelled/segmented data for artifact identification.


In [ ]:
Samples = glob('data/*')
mask_folder = 'masks'

for path_to_sample in Samples:

    path_to_sample = path_to_sample + '/'

    if not os.path.isdir(path_to_sample + mask_folder):
        #Grabbing imctrl file and parsing its contents. If no imctrl file exists, skip this
        try:
            imctrl_file = glob(path_to_sample + '*.imctrl')[0]
        except:
            continue
        else:
            #Assign controls
            controls = parse_imctrl(imctrl_file)

            #Creating mask, need to specify image size here
            #This MASK function was developed from a previous publication by Yanxon et al.
            mask = MASK(controls=controls, shape=(2880,2880))

            #Creating mask directory
            os.mkdir(path_to_sample + mask_folder)
            paths = glob(path_to_sample + '*.tif')

            #Generate masked image for each sample image and save it in a mask subdirectory for a specific dataset
            for path in paths:
                image_name = path.split('/')[-1][:-4]
                image = iio.v2.volread(path)
                result = mask.AutoSpotMask(image, esdmul=7.0)
                iio.v2.imwrite(path_to_sample + f'{mask_folder}/{image_name}_mask.tif', result)


# Creating dataset/dataloaders for training

Here we'll grab put all relevant tif files into a dataset

In [ ]:
#Create dataset object from data files
Samples = [f for f in glob('data/*') if "Nickel" in f or "battery5" in f]
dataset = Dataset(n=len(Samples))

#Grab all tif files from directories in Samples
dataset.get_data(Samples, label_ext = '.tif')



In [ ]:
#Check how much available memory exists 
#Note: This is not the same as the amount of memory used by the dataset
#This is the amount of memory available for the dataset to use


In [ ]:
i = 1
dataset.images[i].size * dataset.images[i].itemsize / 1e6 #size of dataset images in megabytes

Next, we need to define our model parameters. This includes quilter params (for stitching pytorch tensors), and model params (for the UNet).

The purpose of a quilter here is to cut up a large tensor into smaller overlapping pieces which can then be fed into a GPU card (in case there are memory limitations).

Here, we have:<br>
*N* = window size (of a patch)<br>
*M* = Step size (how much the patch/window is moved)<br>
*B* = Border size (Border of window which is multiplied by border weight)<br>

E.g. A window size of 256, step size of 128, with a border size of 32 means that
we take patches of size (256-64 = 192) x 192. These 192 x 192 blocks overlap with one another, since we only take a step of size 128 when moving to a new patch.

In [ ]:
#Define quilter hyperparams
#Quilter takes a large tensor and cuts it up into smaller pieces in case the whole tensor
#does not fit 
N = 256 #Patch size
M = N // 2
B = M // 4

#Define quilter params
quilter_params = {'Y': 2880, 'X': 2880,
                  'window': (N, N),
                  'step': (M, M),
                  'border': (B, B),
                  'border_weight': 0}

#Define TUNet params (from dlsia)
model_params = {'image_shape': (2880, 2880),
                'in_channels': 1,
                'out_channels': 2,
                'base_channels': 8,
                'growth_rate': 2,
                'depth': 4}


Next, we need to define training params as well as define the model

In [ ]:
#Training parameters
epoch = 10
batch_size = 50
lr_rate = 1e-2

model = cmodel(quilter_params, model_params, device = 'cuda:0')

In [ ]:
import random

In [ ]:
dataset.images

In [ ]:
#Train
model.train(dataset, include_data={0: [0],
                                   1: [0],
                                   2: [0],
                                   3: [0],
                                   4: [0],
                                   5: [0]},
            epoch=epoch,
            batch_size=batch_size,
            lr_rate=lr_rate)

model.save('models/test_model.pt')